In [223]:
##importing the required librarires

import pandas as pd
import numpy as np
import datetime as dt
import time
from alpha_vantage.timeseries import TimeSeries


In [224]:
class ScriptData(object):
    
    #initializing constructor 
    def __init__(self):
        self.main_var = pd.DataFrame()
        self.data = {}
    ## get method for cllecting json data 
    def __getJson__(self):
        return self.data
    ## set method for setting json data 
    def __setJson__(self,newkey):
            self.data =newkey
    ## set method for overloading dataframes for diffrent scripts
    def __setDataFrame__(self,newkey):
        self.main_var = newkey
    
    ## get method for getting dataframes for diffrent scripts
    def __getDataFrame__(self):
        return self.main_var
        
    ##this function will fetch data in json format
    def fetch_intraday_data(self,script):
        ts = TimeSeries(key='PBUU95WNPEBUZYZI')
# Get json object with the intraday data and another with  the call's metadata
        data,meta = ts.get_intraday(script)
        self.__setJson__(data)
        print(type(self.__getJson__()))
    
    ##this function will convert json data into pandas data frame
    def convert_intraday_data(self,script):
        df = pd.DataFrame.from_dict(self.data)
    
        df= df.transpose()
        df.reset_index(level = 0, inplace=True)
        df.columns = ["timestamp","open","high","low","close","volume"]

        self.__setDataFrame__(df)## we are setting dataframeto main_varvariableofthe class which willconsist of the main data frame

In [225]:
script_data= ScriptData()## object of class ScriptData
script_data.fetch_intraday_data("GOOGL")
script_data.convert_intraday_data("GOOGL")
script_data.main_var.head()
#script_data.plot(script_data.main_var)

<class 'dict'>


,timestamp,open,high,low,close,volume
0,2022-12-30 20:00:00,88.5200,88.5700,88.4800,88.5500,1828
1,2022-12-30 19:45:00,88.4600,88.5000,88.4500,88.4500,3712
2,2022-12-30 19:30:00,88.4800,88.4800,88.4800,88.4800,244
3,2022-12-30 19:15:00,88.4600,88.4600,88.4500,88.4500,676
4,2022-12-30 19:00:00,88.4200,88.4200,88.4000,88.4000,1984


In [226]:
###calculating moving average
### function to calculate moving average on close data point
def indicator1(df,timeperiod):
    temp= pd.DataFrame()
    temp["close"] = df["close"]
    temp["MA"] = temp["close"].rolling(timeperiod).mean()
    indicator_df = pd.DataFrame()
    indicator_df["timestamp"] = df["timestamp"]
    indicator_df["indicator"] = temp["MA"]
    return indicator_df

indicator1(script_data.main_var,5)

,timestamp,indicator
0,2022-12-30 20:00:00,NaN
1,2022-12-30 19:45:00,NaN
2,2022-12-30 19:30:00,NaN
3,2022-12-30 19:15:00,NaN
4,2022-12-30 19:00:00,88.46600
...,...,...
95,2022-12-29 12:00:00,88.49502
96,2022-12-29 11:45:00,88.49070
97,2022-12-29 11:30:00,88.44370
98,2022-12-29 11:15:00,88.31968


In [227]:
### defining our strategy 

In [252]:
class Strategy:
    def __init__(self,script):
        self.script = script
    ##fetching the required data
    def get_script_data(self):
        script_data.fetch_intraday_data(self.script)
        script_data.convert_intraday_data(self.script)
    
    ##function to geerate signals
    def get_signal(self):
        signals = pd.DataFrame()
        signals["timestamp"] = script_data.main_var["timestamp"]
        signals["signal"] = None
        close_data = script_data.main_var["close"]
        indicator_data = indicator1(script_data.main_var,5)["indicator"]
        temp = pd.DataFrame()###temproray dtaframe
        temp["prev_indicator"] = indicator_data.shift(1)
        temp["indicator"] = indicator_data
        temp["close"] = close_data
        temp = temp.astype(float)

            
        prev_indicator = indicator_data.shift(1)##prev indicator value in column
        prev_close = temp["close"].shift(1)##prev closevalue in column
        
        
        
        ##defining cut_upwards , cut_downward, 
        ## if the current indicator value is greater than close and previous indicator value is less then previous the it is a "BULLISH" crossover
        ##if the current indicator value is less than close and previous indicator value is greater then previous the it is a "BEARISH" crossover

        upward_crossing  = ((temp['indicator'] > temp['close']) & (prev_indicator < prev_close))## returns true if cndition is matced
        downward_crossing = ((temp['indicator'] < temp['close']) & (prev_indicator > prev_close))      
        

        temp["upward_crossing"] = upward_crossing
        temp["downward_crossing"]= downward_crossing
        #print(signals) 
        ## here we are assigning BUY and sell value if the value of upward_crossing& downward_Crossing is TRUE
        for i in range(len(temp["upward_crossing"])):
            if temp["upward_crossing"][i] == True:
                signals["signal"][i] = "BUY"
        for i in range(len(temp["downward_crossing"])):
            if temp["downward_crossing"][i] == True:
                signals["signal"][i] = "SELL"
            #if there is no buy and sell seignal we assign that column with "NO SISGNAL"
            elif signals["signal"][i] == None:
                signals["signal"][i] = "NO SIGNAL"
                
        signal_lst = ["BUY","SELL"]
        #print(temp)
        ##returning the dataframe with only buy and sell value 
        return signals[signals.signal.isin(signal_lst)]
        
        
strat = Strategy("GOOGL")
strat.get_script_data()
strat.get_signal()

<class 'dict'>


,timestamp,signal
8,2022-12-30 18:00:00,SELL
9,2022-12-30 17:45:00,BUY
10,2022-12-30 17:30:00,SELL
11,2022-12-30 17:15:00,BUY
12,2022-12-30 17:00:00,SELL
16,2022-12-30 16:00:00,BUY
24,2022-12-30 14:00:00,SELL
25,2022-12-30 13:45:00,BUY
27,2022-12-30 13:15:00,SELL
31,2022-12-30 12:15:00,BUY


In [229]:
#!pip install pyalgotrading

In [230]:
from pyalgotrading.utils.candlesticks.heikinashi import HeikinAshi
df = script_data.main_var
df['timestamp'] = df['timestamp'].astype('datetime64[ns]')
df = df.astype({col: float for col in df.columns[1:]})



In [231]:
HeikinAshi(df)


,timestamp,open,high,low,close
0,2022-12-30 20:00:00,88.535000,88.570000,88.480,88.53000
1,2022-12-30 19:45:00,88.532500,88.532500,88.450,88.46500
2,2022-12-30 19:30:00,88.498750,88.498750,88.480,88.48000
3,2022-12-30 19:15:00,88.489375,88.489375,88.450,88.45500
4,2022-12-30 19:00:00,88.472187,88.472187,88.400,88.41000
...,...,...,...,...,...
95,2022-12-29 12:00:00,88.518225,88.630000,88.335,88.45250
96,2022-12-29 11:45:00,88.485363,88.485363,88.130,88.25085
97,2022-12-29 11:30:00,88.368106,88.368106,88.005,88.09750
98,2022-12-29 11:15:00,88.232803,88.430000,87.980,88.20000


In [253]:
from pyalgotrading.utils import func
from pyalgotrading.constants import PlotType

indicator_data = indicator1(script_data.main_var,5)["indicator"]

func.plot_candlestick_chart(df,PlotType.HEIKINASHI)

In [254]:
#!pip freeze > requirements.txt